Before running the script, the visdom visualization tool must be initialized. Follow the following instructions:

* Open a new PowerShell or Bash window
* Go to the project repository
* Activate the python environment: `.\.venv\Scripts\activate`
* Execute: `python -m visdom.server`

In [ ]:
#!pip install tifffile
#!pip install einops
#!pip install visdom
#!pip install scikit-image
#!pip install torchinfo
import os
import time
import copy
import torch
import gc
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from models.iqa_module_baseline import mymodel
from utils.dataset import MyDataset_xinbo
import visdom
from random import sample
from scipy.stats import pearsonr, spearmanr, kendalltau
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
from torchinfo import summary

In [ ]:
import random
def seed_torch(seed):
    random.seed(seed) # python seed
    os.environ['PYTHONHASHSEED'] = str(seed) # 设置python哈希种子，for certain hash-based operations (e.g., the item order in a set or a dict）。seed为0的时候表示不用这个feature，也可以设置为整数。 有时候需要在终端执行，到脚本实行可能就迟了。
    np.random.seed(seed) # If you or any of the libraries you are using rely on NumPy, 比如Sampling，或者一些augmentation。 哪些是例外可以看https://pytorch.org/docs/stable/notes/randomness.html
    torch.manual_seed(seed) # 为当前CPU设置随机种子。 pytorch官网倒是说(both CPU and CUDA)
    torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
    # torch.cuda.manual_seed_all(seed) # 使用多块GPU时，均设置随机种子
    torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True # 设置为True时，cuDNN使用非确定性算法寻找最高效算法
    # torch.backends.cudnn.enabled = True # pytorch使用CUDANN加速，即使用GPU加速
seed_torch(seed=1)

In [ ]:
model = mymodel()
model = model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

# Print summary
summary(model,
        input_size=(1,3,480, 270),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
# Define some constants
NUM_WORKERS = 0
AMOUNT_TO_GET = 1.0
SEED = 42
BATCH_SIZE = 2
N_SPLITS = 8
NUM_EPOCHS = 80
ACCUM_STEPS = 2

# Define target data directory
BASELINE_NAME = f"VCIP_IMQA/VCIP"
BASELINE = Path(BASELINE_NAME)
TARGET_DIR = BASELINE / "EQ420_image"
TARGET_LABEL = BASELINE / "Labels"
TARGET_BASE = BASELINE / "IMQA"
TARGET_DIR = r'VCIP_IMQA/VCIP/EQ420_image/'

# Create target model directory
MODEL_DIR = Path("outputs")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Read train csv
idx_csv = pd.read_csv(TARGET_LABEL / 'mos_fold_train.csv').sample(frac=1)

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Check out splitting
fold_vector = [1, 2, 3, 4, 5, 6, 7, 8]
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(kf.split(fold_vector)):
    print(train_idx, val_idx)

#df = pd.DataFrame(0, index=[f"folder {i}" for i in fold_vector], columns=['plcc', 'srocc'])
#df = pd.read_csv(f"plcc_srocc_baseline_folds{N_SPLITS}.csv")
df = pd.DataFrame(0, index=[f"folder {i}" for i in fold_vector] + ["average"] + ["global"], columns=['plcc', 'srocc', 'rmse'])

#display(df.head(11))

for error in ['mse']: #huber
    for opt in ['adam']: #, 'adamw']:
        
        # Initialize result dataframe
        df = pd.DataFrame(0, index=[f"folder {i}" for i in fold_vector] + ["average"] + ["global"], columns=['plcc', 'srocc', 'rmse'])
        csv_file_name = f"plcc_srocc_baseline_folds{N_SPLITS}_{opt}_{error}.csv"

        # Execute K-folds
        all_preds = []
        all_gtmos = []
        kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
        for fold, (train_idx, val_idx) in enumerate(kf.split(fold_vector)):

            #if (fold + 1) <= 3:
            #    continue

            # K-fold preparation
            fold_train = [fold_vector[i] for i in train_idx]
            fold_val = fold_vector[val_idx[0]]
            train_ids = idx_csv[idx_csv['folds'].isin(fold_train)]
            val_ids = idx_csv.loc[idx_csv['folds'] == fold_val]

            
            train_set = MyDataset_xinbo(ids=train_ids, ref_dir=TARGET_DIR)
            val_set = MyDataset_xinbo(ids=val_ids, ref_dir=TARGET_DIR)
            dataloaders = {'train':DataLoader(train_set, batch_size=BATCH_SIZE,shuffle=True, num_workers=4)
                        ,'val':DataLoader(val_set, batch_size=BATCH_SIZE,shuffle=False, num_workers=4)}
            
            dataset_sizes = {'train':len(train_ids),'val':len(val_ids)}
            
            model = mymodel()
            model = model.to(device)

            port = 8097
            viz = visdom.Visdom(port=port)
            win = viz.scatter(X=np.asarray([[0,0]]))    
            viz2 = visdom.Visdom(port=port)    
            viz2.line(np.asarray([[0,0]]), np.asarray([[0,0]]), win='total_loss', opts=dict(title='total_loss', legend=['val', 'train']))
            viz_line = visdom.Visdom(port=port)
            viz_line.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
            viz_line_v = visdom.Visdom(port=port)
            viz_line_v.line([0.], [0.], win='val_loss', opts=dict(title='val loss'))

            if opt == 'adam':
                optimizer = optim.Adam(model.parameters(),lr=5e-6)
            else:
                optimizer = optim.AdamW(model.parameters(),lr=1e-3)
            
            scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

            # criterion1 = create_lossfunc()
            if error == 'mse':
                err = nn.MSELoss()
            else:
                err = nn.HuberLoss()
            #bce = nn.BCELoss()
            #HuberLoss = HuberLoss()

            '''Training'''
            best_model_wts = copy.deepcopy(model.state_dict())
            best_loss = 1000000

            #for k,v in model.named_parameters():
            #    print('{}: {}'.format(k, v.requires_grad))

            global_step_t = 0
            global_step_v = 0

            for epoch in range(NUM_EPOCHS):
                print('Epoch {}/{}'.format(epoch + 1, NUM_EPOCHS))
                print('-' * 10)

                # Each epoch has a training and validation phase
                for phase in ['train', 'val']:
                    if phase == 'train':
                        model.train()  # Set model to training mode
                    else:
                        model.eval()   # Set model to evaluate mode

                    running_loss = 0.0

                    # Iterate over data
                    optimizer.zero_grad()
                    for i_batch, sample_batched in tqdm(enumerate(dataloaders[phase])):

                        ref, mos = sample_batched['ref'], sample_batched['mos']
                        ref, mos = ref.type(torch.cuda.FloatTensor), mos.type(torch.cuda.FloatTensor)
                        ref, mos = ref.to(device), mos.to(device)

                        # zero the parameter gradients
                        #optimizer.zero_grad()

                        # forward
                        with torch.set_grad_enabled(phase == 'train'):
                            pred_score  = model(ref)
                            if error == 'mse':
                                loss = err(pred_score.squeeze(), mos.squeeze())
                            else:
                                loss = err(pred_score, mos)
                            #loss = HuberLoss(pred_score, mos)

                            loss = loss / ACCUM_STEPS

                            # backward + optimize only if in training phase
                            if phase == 'train':
                                loss.backward()
                                #optimizer.step()

                        # Reset batch accumulation
                        if (i_batch + 1) % ACCUM_STEPS == 0 or (i_batch + 1) == dataset_sizes[phase]:
                            optimizer.step()
                            optimizer.zero_grad()

                        # statistics
                        running_loss += loss.item() * ref.size(0) * ACCUM_STEPS
                        if phase == 'train':  
                            viz_line.line([loss.item()], [global_step_t], win='train_loss', update='append')             
                            global_step_t += 1
                        elif phase == 'val':
                            viz_line_v.line([loss.item()], [global_step_v], win='val_loss', update='append')
                            global_step_v += 1

                    if phase == 'train':
                        scheduler.step()

                    epoch_loss = running_loss / dataset_sizes[phase]

                    if phase == 'train':
                        viz.scatter(X=np.array([[epoch,epoch_loss]]),
                                    name="train",
                                    win=win,
                                    update="append")
                        epoch_loss_v = epoch_loss

                    elif phase == 'val':
                        viz.scatter(X=np.array([[epoch,epoch_loss]]),
                                    name="val",
                                    win=win,
                                    update="append")
                        epoch_loss_t = epoch_loss

                    if phase == 'val':
                        viz2.line(np.asarray([[epoch_loss_t, epoch_loss_v]]), np.asarray([[epoch,epoch]]), win='total_loss', opts=dict(title='total_loss', legend=['val','train']), update='append')

                    #print('{} Loss: {:.4f}'.format(phase, epoch_loss))

                    if phase == 'val' and epoch_loss < best_loss:
                        best_loss = epoch_loss
                        best_model_wts = copy.deepcopy(model.state_dict())
                        counter = 0
                    elif phase == 'val' and epoch_loss >= best_loss:
                        counter += 1
                        if counter ==2:
                            print('early stopped!')
                            break
                else:
                    continue
                break
                print()

            print('Best val loss: {:4f}'.format(best_loss))
            
            val_dataloader = {'val':DataLoader(val_set, batch_size=1,shuffle=False, num_workers=4)}

            model.eval()
            res = []

            for i_batch, sample_batched in tqdm(enumerate(val_dataloader["val"])):
                ref, mos = sample_batched['ref'], sample_batched['mos']
                ref, mos = ref.type(torch.cuda.FloatTensor), mos.type(torch.cuda.FloatTensor)
                ref, mos = ref.to(device), mos.to(device)
                with torch.set_grad_enabled(False):
                    pred_score  = model(ref)
                    pred_score = pred_score.squeeze().cpu().item()
                    res.append(pred_score)
            
            # Compute metrics  
            gtmos = val_ids["mos"].tolist()    
            plcc, _ = pearsonr(res, gtmos)
            srocc, _ = spearmanr(res, gtmos)
            rmse = root_mean_squared_error(res, gtmos)

            # Store metrics
            df.iloc[fold] = [plcc, srocc, rmse] 

            # Accumulate all predictions and ground truths
            all_preds.extend(res)
            all_gtmos.extend(gtmos)

            df.to_csv(csv_file_name,index=False)

            model.load_state_dict(best_model_wts)
            #savepath = './outputs/iqa_total_20250616_' + 'fold' +str(fold) + error + opt
            savepath = MODEL_DIR / f"iqa_total_20250616_fold{fold}{error}{opt}"
            torch.save(model.state_dict(),savepath)

            del model, best_model_wts
            del train_set, val_set, dataloaders, val_dataloader, err, scheduler, optimizer
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.synchronize()

        # Compute average metrics across folds
        mean_plcc = df['plcc'].iloc[:8].mean()
        mean_srocc = df['srocc'].iloc[:8].mean()
        mean_rmse = df['rmse'].iloc[:8].mean()

        # Compute global metrics from all predictions
        global_plcc, _ = pearsonr(all_preds, all_gtmos)
        global_srocc, _ = spearmanr(all_preds, all_gtmos)
        global_rmse = root_mean_squared_error(all_preds, all_gtmos)

        # Add a final row to the DataFrame
        df.loc['average'] = [mean_plcc, mean_srocc, mean_rmse]
        df.loc['global'] = [global_plcc, global_srocc, global_rmse]
        df.to_csv(csv_file_name, index=False)